***Copyright 2020 The TensorFlow Authors.***

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Restricciones de forma por ética con Tensorflow Lattice

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lattice/tutorials/shape_constraints_for_ethics"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/lattice/tutorials/shape_constraints_for_ethics.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/lattice/tutorials/shape_constraints_for_ethics.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/lattice/tutorials/shape_constraints_for_ethics.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>

## Descripción general

En este tutorial se enseña cómo se puede usar la biblioteca TensorFlow Lattice (TFL) para entrenar modelos que se comporten *de manera responsable* y no violen ciertos supuestos que son *éticos* o *justos*. En particular, nos centraremos en el uso de restricciones de monotonicidad para evitar *una penalización injusta* de ciertos atributos. Este tutorial incluye demostraciones de los experimentos del artículo [*Ética deontológica por restricciones de forma de monotonicidad*](https://arxiv.org/abs/2001.11990) de Serena Wang y Maya Gupta, publicado en [AISTATS 2020](https://www.aistats.org/).

Usaremos estimadores prediseñados de TFL en conjuntos de datos públicos, pero tenga en cuenta que todo lo incluido en este tutorial también se puede hacer con modelos construidos a partir de capas TFL Keras.

Antes de continuar, asegúrese de que su tiempo de ejecución tenga instalados todos los paquetes necesarios (tal como se importan en las celdas de código a continuación).

## Preparación

Instalar el paquete TF Lattice:

In [ ]:
#@test {"skip": true}
!pip install tensorflow-lattice tensorflow_decision_forests seaborn

Importar los paquetes requeridos:

In [ ]:
import tensorflow as tf
import tensorflow_lattice as tfl
import tensorflow_decision_forests as tfdf

import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import sys
import tempfile
logging.disable(sys.maxsize)

Valores predeterminados que se usan en este tutorial:

In [ ]:
# List of learning rate hyperparameters to try.
# For a longer list of reasonable hyperparameters, try [0.001, 0.01, 0.1].
LEARNING_RATES = [0.01]
# Default number of training epochs and batch sizes.
NUM_EPOCHS = 1000
BATCH_SIZE = 1000
# Directory containing dataset files.
DATA_DIR = 'https://raw.githubusercontent.com/serenalwang/shape_constraints_for_ethics/master'

# Estudio de caso n.º 1: admisiones a la facultad de derecho

En la primera parte de este tutorial, consideraremos un estudio de caso que usa el conjunto de datos de Admisiones a la Facultad de Derecho del Consejo de Admisiones a la Facultad de Derecho (LSAC). Entrenaremos a un clasificador para predecir si un estudiante aprobará o no el examen mediante el uso de dos características: la calificación LSAT y GPA del estudiante.

Supongamos que la puntuación del clasificador se usara para orientar las admisiones o becas a la facultad de derecho. De acuerdo con las normas sociales basadas en el mérito, esperaríamos que los estudiantes con un GPA más alto y una calificación LSAT más alta recibieran una puntuación más alta del clasificador. Sin embargo, observaremos que es fácil para los modelos violar estas normas intuitivas y, en ocasiones, penalizar a las personas por tener una calificación GPA o LSAT más alta.

Para abordar este problema de *penalización injusta*, podemos imponer restricciones de monocinidad para que un modelo nunca penalice un GPA más alto o una puntuación LSAT más alta, en igualdad de condiciones. En este tutorial, mostraremos cómo imponer esas restricciones de monotonicidad con TFL.

## Cargar datos de la facultad de derecho

In [ ]:
# Load data file.
law_file_name = 'lsac.csv'
law_file_path = os.path.join(DATA_DIR, law_file_name)
raw_law_df = pd.read_csv(law_file_path, delimiter=',')

Preprocesar los conjunto de datos:

In [ ]:
# Define label column name.
LAW_LABEL = 'pass_bar'

In [ ]:
def preprocess_law_data(input_df):
  # Drop rows with where the label or features of interest are missing.
  output_df = input_df[~input_df[LAW_LABEL].isna() & ~input_df['ugpa'].isna() &
                       (input_df['ugpa'] > 0) & ~input_df['lsat'].isna()]
  return output_df


law_df = preprocess_law_data(raw_law_df)

### Dividir los datos en conjuntos de entrenamiento/validación/prueba

In [ ]:
def split_dataset(input_df, random_state=888):
  """Splits an input dataset into train, val, and test sets."""
  train_df, test_val_df = train_test_split(
      input_df, test_size=0.3, random_state=random_state)
  val_df, test_df = train_test_split(
      test_val_df, test_size=0.66, random_state=random_state)
  return train_df, val_df, test_df


law_train_df, law_val_df, law_test_df = split_dataset(law_df)

### Visualizar la distribución de datos

Primero visualizaremos la distribución de los datos. Trazaremos las calificaciones GPA y LSAT para todos los estudiantes que aprobaron el examen y también para todos los estudiantes que no aprobaron el examen.

In [ ]:
def plot_dataset_contour(input_df, title):
  plt.rcParams['font.family'] = ['serif']
  g = sns.jointplot(
      x='ugpa',
      y='lsat',
      data=input_df,
      kind='kde',
      xlim=[1.4, 4],
      ylim=[0, 50])
  g.plot_joint(plt.scatter, c='b', s=10, linewidth=1, marker='+')
  g.ax_joint.collections[0].set_alpha(0)
  g.set_axis_labels('Undergraduate GPA', 'LSAT score', fontsize=14)
  g.fig.suptitle(title, fontsize=14)
  # Adust plot so that the title fits.
  plt.subplots_adjust(top=0.9)
  plt.show()

In [ ]:
law_df_pos = law_df[law_df[LAW_LABEL] == 1]
plot_dataset_contour(
    law_df_pos, title='Distribution of students that passed the bar')

In [ ]:
law_df_neg = law_df[law_df[LAW_LABEL] == 0]
plot_dataset_contour(
    law_df_neg, title='Distribution of students that failed the bar')

## Entrenar un modelo lineal calibrado para predecir la aprobación del examen de admisión

A continuación, entrenaremos un *modelo lineal calibrado* de TFL para predecir si un estudiante aprobará o no el examen. Las dos funciones de entrada serán la calificación LSAT y el GPA, y la etiqueta de entrenamiento será si el estudiante aprobó el examen.

Primero entrenaremos un modelo lineal calibrado sin restricciones. Luego, entrenaremos un modelo lineal calibrado con restricciones de monotonicidad y observaremos la diferencia en el resultado y la precisión del modelo.

### Funciones ayudantes para entrenar un estimador lineal calibrado de TFL

Estas funciones se usarán para este estudio de caso de la facultad de derecho, así como para el estudio de caso de incumplimiento crediticio a continuación.

In [ ]:
def train_tfl_estimator(train_df, monotonicity, learning_rate, num_epochs,
                        batch_size, get_input_fn,
                        get_feature_columns_and_configs):
  """Trains a TFL calibrated linear estimator.

  Args:
    train_df: pandas dataframe containing training data.
    monotonicity: if 0, then no monotonicity constraints. If 1, then all
      features are constrained to be monotonically increasing.
    learning_rate: learning rate of Adam optimizer for gradient descent.
    num_epochs: number of training epochs.
    batch_size: batch size for each epoch. None means the batch size is the full
      dataset size.
    get_input_fn: function that returns the input_fn for a TF estimator.
    get_feature_columns_and_configs: function that returns TFL feature columns
      and configs.

  Returns:
    estimator: a trained TFL calibrated linear estimator.

  """
  feature_columns, feature_configs = get_feature_columns_and_configs(
      monotonicity)

  model_config = tfl.configs.CalibratedLinearConfig(
      feature_configs=feature_configs, use_bias=False)

  estimator = tfl.estimators.CannedClassifier(
      feature_columns=feature_columns,
      model_config=model_config,
      feature_analysis_input_fn=get_input_fn(input_df=train_df, num_epochs=1),
      optimizer=tf.keras.optimizers.legacy.Adam(learning_rate))

  estimator.train(
      input_fn=get_input_fn(
          input_df=train_df, num_epochs=num_epochs, batch_size=batch_size))
  return estimator


def optimize_learning_rates(
    train_df,
    val_df,
    test_df,
    monotonicity,
    learning_rates,
    num_epochs,
    batch_size,
    get_input_fn,
    get_feature_columns_and_configs,
):
  """Optimizes learning rates for TFL estimators.

  Args:
    train_df: pandas dataframe containing training data.
    val_df: pandas dataframe containing validation data.
    test_df: pandas dataframe containing test data.
    monotonicity: if 0, then no monotonicity constraints. If 1, then all
      features are constrained to be monotonically increasing.
    learning_rates: list of learning rates to try.
    num_epochs: number of training epochs.
    batch_size: batch size for each epoch. None means the batch size is the full
      dataset size.
    get_input_fn: function that returns the input_fn for a TF estimator.
    get_feature_columns_and_configs: function that returns TFL feature columns
      and configs.

  Returns:
    A single TFL estimator that achieved the best validation accuracy.
  """
  estimators = []
  train_accuracies = []
  val_accuracies = []
  test_accuracies = []
  for lr in learning_rates:
    estimator = train_tfl_estimator(
        train_df=train_df,
        monotonicity=monotonicity,
        learning_rate=lr,
        num_epochs=num_epochs,
        batch_size=batch_size,
        get_input_fn=get_input_fn,
        get_feature_columns_and_configs=get_feature_columns_and_configs)
    estimators.append(estimator)
    train_acc = estimator.evaluate(
        input_fn=get_input_fn(train_df, num_epochs=1))['accuracy']
    val_acc = estimator.evaluate(
        input_fn=get_input_fn(val_df, num_epochs=1))['accuracy']
    test_acc = estimator.evaluate(
        input_fn=get_input_fn(test_df, num_epochs=1))['accuracy']
    print('accuracies for learning rate %f: train: %f, val: %f, test: %f' %
          (lr, train_acc, val_acc, test_acc))
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)
    test_accuracies.append(test_acc)
  max_index = val_accuracies.index(max(val_accuracies))
  return estimators[max_index]

### Funciones ayudantes para configurar las funciones del conjunto de datos de la facultad de derecho

Estas funciones ayudantes son específicas del estudio de caso de la facultad de derecho.

In [ ]:
def get_input_fn_law(input_df, num_epochs, batch_size=None):
  """Gets TF input_fn for law school models."""
  return tf.compat.v1.estimator.inputs.pandas_input_fn(
      x=input_df[['ugpa', 'lsat']],
      y=input_df['pass_bar'],
      num_epochs=num_epochs,
      batch_size=batch_size or len(input_df),
      shuffle=False)


def get_feature_columns_and_configs_law(monotonicity):
  """Gets TFL feature configs for law school models."""
  feature_columns = [
      tf.feature_column.numeric_column('ugpa'),
      tf.feature_column.numeric_column('lsat'),
  ]
  feature_configs = [
      tfl.configs.FeatureConfig(
          name='ugpa',
          lattice_size=2,
          pwl_calibration_num_keypoints=20,
          monotonicity=monotonicity,
          pwl_calibration_always_monotonic=False),
      tfl.configs.FeatureConfig(
          name='lsat',
          lattice_size=2,
          pwl_calibration_num_keypoints=20,
          monotonicity=monotonicity,
          pwl_calibration_always_monotonic=False),
  ]
  return feature_columns, feature_configs

### Funciones ayudantes para la visualización de resultados del modelo entrenado

In [ ]:
def get_predicted_probabilities(estimator, input_df, get_input_fn):
  if isinstance(estimator, tf.estimator.Estimator):
    predictions = estimator.predict(
        input_fn=get_input_fn(input_df=input_df, num_epochs=1))
    return [prediction['probabilities'][1] for prediction in predictions]
  else:
    return estimator.predict(tfdf.keras.pd_dataframe_to_tf_dataset(input_df))


def plot_model_contour(estimator, input_df, num_keypoints=20):
  x = np.linspace(min(input_df['ugpa']), max(input_df['ugpa']), num_keypoints)
  y = np.linspace(min(input_df['lsat']), max(input_df['lsat']), num_keypoints)

  x_grid, y_grid = np.meshgrid(x, y)

  positions = np.vstack([x_grid.ravel(), y_grid.ravel()])
  plot_df = pd.DataFrame(positions.T, columns=['ugpa', 'lsat'])
  plot_df[LAW_LABEL] = np.ones(len(plot_df))
  predictions = get_predicted_probabilities(
      estimator=estimator, input_df=plot_df, get_input_fn=get_input_fn_law)
  grid_predictions = np.reshape(predictions, x_grid.shape)

  plt.rcParams['font.family'] = ['serif']
  plt.contour(
      x_grid,
      y_grid,
      grid_predictions,
      colors=('k',),
      levels=np.linspace(0, 1, 11))
  plt.contourf(
      x_grid,
      y_grid,
      grid_predictions,
      cmap=plt.cm.bone,
      levels=np.linspace(0, 1, 11))  # levels=np.linspace(0,1,8));
  plt.xticks(fontsize=20)
  plt.yticks(fontsize=20)

  cbar = plt.colorbar()
  cbar.ax.set_ylabel('Model score', fontsize=20)
  cbar.ax.tick_params(labelsize=20)

  plt.xlabel('Undergraduate GPA', fontsize=20)
  plt.ylabel('LSAT score', fontsize=20)

## Entrenar un modelo lineal calibrado sin restricciones (sin monoticinidad)

In [ ]:
nomon_linear_estimator = optimize_learning_rates(
    train_df=law_train_df,
    val_df=law_val_df,
    test_df=law_test_df,
    monotonicity=0,
    learning_rates=LEARNING_RATES,
    batch_size=BATCH_SIZE,
    num_epochs=NUM_EPOCHS,
    get_input_fn=get_input_fn_law,
    get_feature_columns_and_configs=get_feature_columns_and_configs_law)

In [ ]:
plot_model_contour(nomon_linear_estimator, input_df=law_df)

## Entrenar modelo lineal calibrado con monoticinidad

In [ ]:
mon_linear_estimator = optimize_learning_rates(
    train_df=law_train_df,
    val_df=law_val_df,
    test_df=law_test_df,
    monotonicity=1,
    learning_rates=LEARNING_RATES,
    batch_size=BATCH_SIZE,
    num_epochs=NUM_EPOCHS,
    get_input_fn=get_input_fn_law,
    get_feature_columns_and_configs=get_feature_columns_and_configs_law)

In [ ]:
plot_model_contour(mon_linear_estimator, input_df=law_df)

## Entrenar otros modelos sin restricciones

Demostramos que los modelos lineales calibrados de TFL se podían entrenar para que tengan monoticinidad tanto en la calificación LSAT como en el GPA sin sacrificar demasiado la precisión.

Pero, ¿cómo se compara el modelo lineal calibrado con otros tipos de modelos, como las redes neuronales profundas (DNN) o los árboles potenciados por gradiente (GBT)? ¿Parece que las DNN y los GBT tienen resultados razonablemente justos? Para abordar esta pregunta, a continuación entrenaremos un DNN y un GBT sin restricciones. De hecho, observaremos que tanto el DNN como el GBT violan fácilmente la monotonicidad en la calificación LSAT y el GPA.

### Entrenar un modelo de red neuronal profunda (DNN) sin restricciones

La arquitectura se optimizó previamente para lograr una alta precisión de validación.

In [ ]:
feature_names = ['ugpa', 'lsat']

dnn_estimator = tf.estimator.DNNClassifier(
    feature_columns=[
        tf.feature_column.numeric_column(feature) for feature in feature_names
    ],
    hidden_units=[100, 100],
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.008),
    activation_fn=tf.nn.relu)

dnn_estimator.train(
    input_fn=get_input_fn_law(
        law_train_df, batch_size=BATCH_SIZE, num_epochs=NUM_EPOCHS))
dnn_train_acc = dnn_estimator.evaluate(
    input_fn=get_input_fn_law(law_train_df, num_epochs=1))['accuracy']
dnn_val_acc = dnn_estimator.evaluate(
    input_fn=get_input_fn_law(law_val_df, num_epochs=1))['accuracy']
dnn_test_acc = dnn_estimator.evaluate(
    input_fn=get_input_fn_law(law_test_df, num_epochs=1))['accuracy']
print('accuracies for DNN: train: %f, val: %f, test: %f' %
      (dnn_train_acc, dnn_val_acc, dnn_test_acc))

In [ ]:
plot_model_contour(dnn_estimator, input_df=law_df)

### Entrenar un modelo de árboles potenciados por gradiente (GBT) sin restricciones

La estructura de árbol se optimizó previamente para lograr una alta precisión de validación.

In [ ]:
law_train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
    law_train_df, label='pass_bar')
law_test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
    law_test_df, label='pass_bar')
law_val_ds = tfdf.keras.pd_dataframe_to_tf_dataset(law_val_df, label='pass_bar')

tree_model = tfdf.keras.GradientBoostedTreesModel(
    features=[tfdf.keras.FeatureUsage(name=name) for name in feature_names],
    exclude_non_specified_features=True,
    num_threads=1,
    num_trees=20,
    max_depth=4,
    growing_strategy='BEST_FIRST_GLOBAL',
    random_seed=42,
    temp_directory=tempfile.mkdtemp(),
)
tree_model.compile(metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')])
tree_model.fit(law_train_ds, validation_data=law_val_ds, verbose=0)

tree_train_acc = tree_model.evaluate(law_train_ds, verbose=0)[1]
tree_val_acc = tree_model.evaluate(law_val_ds, verbose=0)[1]
tree_test_acc = tree_model.evaluate(law_test_ds, verbose=0)[1]
print('accuracies for GBT: train: %f, val: %f, test: %f' %
      (tree_train_acc, tree_val_acc, tree_test_acc))

In [ ]:
plot_model_contour(tree_model, input_df=law_df)

# Estudio de caso n.º 2: incumplimiento crediticio

El segundo estudio de caso que consideraremos en este tutorial es la predicción de la probabilidad de incumplimiento crediticio de un individuo. Usaremos el conjunto de datos de incumplimiento de clientes de tarjetas de crédito del repositorio de UCI. Estos datos se recopilaron de 30.000 usuarios de tarjetas de crédito taiwaneses y contienen una etiqueta binaria que indica si un usuario incumplió o no un pago en un período de tiempo. Las características incluyen estado civil, género, educación y cuánto tiempo está atrasado un usuario en el pago de sus facturas existentes, para cada uno de los meses de abril a septiembre de 2005.

Como hicimos con el primer estudio de caso, ilustramos nuevamente el uso de restricciones de monotonicidad para evitar *penalizaciones injustas*: si el modelo se usara para determinar el puntaje crediticio de un usuario, muchos podrían considerar injusto si fueran penalizados por pagar sus facturas antes y todo lo demás sigue igual. Por lo tanto, aplicamos una restricción de monotonicidad que evita que el modelo penalice los pagos anticipados.

## Cargar los datos de incumplimiento crediticio

In [ ]:
# Load data file.
credit_file_name = 'credit_default.csv'
credit_file_path = os.path.join(DATA_DIR, credit_file_name)
credit_df = pd.read_csv(credit_file_path, delimiter=',')

In [ ]:
# Define label column name.
CREDIT_LABEL = 'default'

### Dividir los datos en conjuntos de entrenamiento/validación/prueba

In [ ]:
credit_train_df, credit_val_df, credit_test_df = split_dataset(credit_df)

### Visualizar la distribución de datos

Primero visualizaremos la distribución de los datos. Trazaremos la media y el error estándar de la tasa de incumplimiento observada para personas con diferentes estados civiles y estados de pago. El estado de pago representa el número de meses que una persona está atrasada en el pago de su préstamo (a abril de 2005).

In [ ]:
def get_agg_data(df, x_col, y_col, bins=11):
  xbins = pd.cut(df[x_col], bins=bins)
  data = df[[x_col, y_col]].groupby(xbins).agg(['mean', 'sem'])
  return data


def plot_2d_means_credit(input_df, x_col, y_col, x_label, y_label):
  plt.rcParams['font.family'] = ['serif']
  _, ax = plt.subplots(nrows=1, ncols=1)
  plt.setp(ax.spines.values(), color='black', linewidth=1)
  ax.tick_params(
      direction='in', length=6, width=1, top=False, right=False, labelsize=18)
  df_single = get_agg_data(input_df[input_df['MARRIAGE'] == 1], x_col, y_col)
  df_married = get_agg_data(input_df[input_df['MARRIAGE'] == 2], x_col, y_col)
  ax.errorbar(
      df_single[(x_col, 'mean')],
      df_single[(y_col, 'mean')],
      xerr=df_single[(x_col, 'sem')],
      yerr=df_single[(y_col, 'sem')],
      color='orange',
      marker='s',
      capsize=3,
      capthick=1,
      label='Single',
      markersize=10,
      linestyle='')
  ax.errorbar(
      df_married[(x_col, 'mean')],
      df_married[(y_col, 'mean')],
      xerr=df_married[(x_col, 'sem')],
      yerr=df_married[(y_col, 'sem')],
      color='b',
      marker='^',
      capsize=3,
      capthick=1,
      label='Married',
      markersize=10,
      linestyle='')
  leg = ax.legend(loc='upper left', fontsize=18, frameon=True, numpoints=1)
  ax.set_xlabel(x_label, fontsize=18)
  ax.set_ylabel(y_label, fontsize=18)
  ax.set_ylim(0, 1.1)
  ax.set_xlim(-2, 8.5)
  ax.patch.set_facecolor('white')
  leg.get_frame().set_edgecolor('black')
  leg.get_frame().set_facecolor('white')
  leg.get_frame().set_linewidth(1)
  plt.show()

In [ ]:
plot_2d_means_credit(credit_train_df, 'PAY_0', 'default',
                     'Repayment Status (April)', 'Observed default rate')

## Entrenar un modelo lineal calibrado para predecir la tasa de incumplimiento crediticio

A continuación, entrenaremos un *modelo lineal calibrado* de TFL para predecir si una persona incumplirá o no con un préstamo. Las dos características de entrada serán el estado civil de la persona y cuántos meses está atrasada en el pago de sus préstamos en abril (estado de pago). La etiqueta de entrenamiento será si la persona incumplió o no con un préstamo.

Primero entrenaremos un modelo lineal calibrado sin restricciones. Luego, entrenaremos un modelo lineal calibrado con restricciones de monotonicidad y observaremos la diferencia en el resultado y la precisión del modelo.

### Funciones ayudantes para configurar las funciones del conjunto de datos crediticios predeterminado

Estas funciones ayudantes son específicas del estudio de caso de incumplimiento crediticio.

In [ ]:
def get_input_fn_credit(input_df, num_epochs, batch_size=None):
  """Gets TF input_fn for credit default models."""
  return tf.compat.v1.estimator.inputs.pandas_input_fn(
      x=input_df[['MARRIAGE', 'PAY_0']],
      y=input_df['default'],
      num_epochs=num_epochs,
      batch_size=batch_size or len(input_df),
      shuffle=False)


def get_feature_columns_and_configs_credit(monotonicity):
  """Gets TFL feature configs for credit default models."""
  feature_columns = [
      tf.feature_column.numeric_column('MARRIAGE'),
      tf.feature_column.numeric_column('PAY_0'),
  ]
  feature_configs = [
      tfl.configs.FeatureConfig(
          name='MARRIAGE',
          lattice_size=2,
          pwl_calibration_num_keypoints=3,
          monotonicity=monotonicity,
          pwl_calibration_always_monotonic=False),
      tfl.configs.FeatureConfig(
          name='PAY_0',
          lattice_size=2,
          pwl_calibration_num_keypoints=10,
          monotonicity=monotonicity,
          pwl_calibration_always_monotonic=False),
  ]
  return feature_columns, feature_configs

### Funciones ayudantes para la visualización de resultados del modelo entrenado

In [ ]:
def plot_predictions_credit(input_df,
                            estimator,
                            x_col,
                            x_label='Repayment Status (April)',
                            y_label='Predicted default probability'):
  predictions = get_predicted_probabilities(
      estimator=estimator, input_df=input_df, get_input_fn=get_input_fn_credit)
  new_df = input_df.copy()
  new_df.loc[:, 'predictions'] = predictions
  plot_2d_means_credit(new_df, x_col, 'predictions', x_label, y_label)

## Entrenar un modelo lineal calibrado sin restricciones (sin monoticinidad)

In [ ]:
nomon_linear_estimator = optimize_learning_rates(
    train_df=credit_train_df,
    val_df=credit_val_df,
    test_df=credit_test_df,
    monotonicity=0,
    learning_rates=LEARNING_RATES,
    batch_size=BATCH_SIZE,
    num_epochs=NUM_EPOCHS,
    get_input_fn=get_input_fn_credit,
    get_feature_columns_and_configs=get_feature_columns_and_configs_credit)

In [ ]:
plot_predictions_credit(credit_train_df, nomon_linear_estimator, 'PAY_0')

## Entrenar modelo lineal calibrado con monoticinidad

In [ ]:
mon_linear_estimator = optimize_learning_rates(
    train_df=credit_train_df,
    val_df=credit_val_df,
    test_df=credit_test_df,
    monotonicity=1,
    learning_rates=LEARNING_RATES,
    batch_size=BATCH_SIZE,
    num_epochs=NUM_EPOCHS,
    get_input_fn=get_input_fn_credit,
    get_feature_columns_and_configs=get_feature_columns_and_configs_credit)

In [ ]:
plot_predictions_credit(credit_train_df, mon_linear_estimator, 'PAY_0')